# ETH Shock Analysis

## Purpose
Expore how diffenent statues(liquidation ratios, liquidity, surplus, etc) affect the risk of bad-debt creation in the Reflexer system by simulating *instantaneous* ETH/USD shocks.
To examine historical ETH shocks see [Historical ETH Shocks](<Historical ETH Shocks.ipynb>)

In [ ]:
import time
import math
import pandas as pd
import numpy as np
import requests
import json
import multiprocess
from multiprocess import Pool
from decimal import Decimal
import matplotlib.pyplot as plt
import scipy.stats as ss
from sklearn.preprocessing import PowerTransformer
from web3 import Web3, HTTPProvider

import seaborn as sns
from fitter import Fitter, get_common_distributions, get_distributions

from uniswap import get_input_price, get_output_price, buy_to_price
from graph_util import fetch_safes, fetch_rp, fetch_debt_ceiling, fetch_saviour_safes
from web3_util import fetch_saviour_targets
from util import chunks

In [ ]:
plt.rcParams['figure.figsize'] = [12, 8]
plt.rcParams['figure.dpi'] = 100 

## Define Some Sim Functions

In [ ]:
def update_cratios(safes, eth_price, redemption_price):
    # update c-ratio based on `eth_price` and `redemption_price`
    new_safes = safes.copy()
    new_safes['cratio'] = new_safes['collateral'] * eth_price / (new_safes['debt'] * redemption_price)
    return new_safes

In [ ]:
def liquidate_critical(safes, liq_cratio=1.40):
    # Return amount of collateral/debt liquidated
    critical_safes = safes.query(f'cratio < {liq_cratio}')

    critical_collateral = critical_safes['collateral'].sum()
    critical_debt = critical_safes['debt'].sum()

    return critical_collateral, critical_debt

def adjust_safe_debt_and_cratios(safes, debt_adj, eth_price, redemption_price):
    # adjust debt by `debt_adj` multiplier
    if debt_adj == 1:
        return safes
    new_safes = safes.copy()
    
    new_safes['debt'] *= debt_adj

    return update_cratios(new_safes, eth_price, redemption_price)

## Fetch Data

In [ ]:
#graphql_url = 'https://subgraph.reflexer.finance/subgraphs/name/reflexer-labs/rai'
graphql_url = 'https://api.thegraph.com/subgraphs/name/reflexer-labs/rai-mainnet'
eth_usd_url = 'https://api.coingecko.com/api/v3/simple/price?ids=ethereum&vs_currencies=usd'
ETH_RPC_URL = 'https://mainnet.infura.io/v3/71de86fb79e940a7970bad3f43c0da4e'

web3 = Web3(HTTPProvider(endpoint_uri=ETH_RPC_URL))

ETH_USD = requests.get(eth_usd_url).json()['ethereum']['usd']
REDEMPTION_PRICE = fetch_rp(graphql_url)
DEBT_CEILING = fetch_debt_ceiling(graphql_url)

# get saviour safes from graph
saviour_safes = fetch_saviour_safes(graphql_url)

# get LP info not available in graph
saviour_safes = fetch_saviour_targets(web3, saviour_safes)

orig_safes = fetch_safes(graphql_url)

In [ ]:
orig_safes.head(5)

In [ ]:
print(update_cratios(saviour_safes, ETH_USD, REDEMPTION_PRICE).head(50))

In [ ]:
# only consider SAFEs with debt
orig_safes = orig_safes.query('debt > 0').reset_index(drop=True)
saviour_safes = saviour_safes.query('debt > 0').reset_index(drop=True)

# Extra debt and collateral that can be created if debt ceiling is exhausted(use system c-ratio)
extra_debt = DEBT_CEILING  - orig_safes['debt'].sum()

In [ ]:
orig_safes['safeId'] = orig_safes['safeId'].astype(int)
saviour_safes['safeId'] = saviour_safes['safeId'].astype(int)

In [ ]:
both_safes = pd.merge(orig_safes, saviour_safes, on='safeId', how='outer', indicator=True)
non_saviour_safes = both_safes.query('_merge == "left_only"').reset_index(drop=True)\
                    .rename({'collateral_x': 'collateral', 'debt_x': 'debt'}, axis=1)\
                    [['collateral', 'debt', 'safeId']]
print(non_saviour_safes['collateral'].sum(), orig_safes['collateral'].sum())

## Mainnet Parameters

In [ ]:
MAINNET_LIQ_RATIO = 1.40 
LIQ_PENALTY = 1.10
INITIAL_SURPLUS = 491000
MIN_DISCOUNT = 0.92
MAX_DISCOUNT = 0.90

# use max_discount for collateral_auctions
DISCOUNT = MAX_DISCOUNT

# What percent of SAFE owners avoid liquidation with OSM delay
OSM_RESPONSIVENESS = 0.35 

# Pct of collateral w/ saviours
SAVIOUR_COLLATERAL = 0.15

# RAI pools
RAI_ETH_V2= 5.4E6
RAI_ETH_V3 = 26.3E3 
RAI_DAI_V3 = 1.38E6 
RAI_USDC_V3 = 16.6E3 

rai_v2_pool = RAI_ETH_V2 + RAI_ETH_V3
usd_v2_pool = rai_v2_pool * REDEMPTION_PRICE

rai_v3_pool = RAI_DAI_V3 + RAI_USDC_V3
usd_v3_pool = rai_v3_pool * REDEMPTION_PRICE

#### Not currently used
SURPLUS_BUFFER = 500000 # Not currently used
DISCOUNT_UPDATE_RATE = Decimal(0.999991859697312485818842992) # Not currently used
LIQUIDATION_QUANTITY = 90000.00 # Not currently used
ON_AUCTION_SYSTEM_COIN_LIMIT = 4025086.73 # Not currently used
#####


## Sim Parameters

In [ ]:
N_RUNS = 10
LRS = [1.20, 1.25, 1.30, 1.35, 1.40]
LRS = [1.30, 1.35]
SURPLUSES = [INITIAL_SURPLUS, 1E6]

min_shock = 0.01
max_shock = 0.6
n_shocks = 61
SHOCKS = np.round(np.linspace(min_shock, max_shock, n_shocks), 4)
TOTAL_SHOCKS = [1,2,3]

# Analysis Parameters
QUANTILES = [0.90, 0.95, 0.99]

PLOT = False
VERBOSE = False

## Overview

Explore how system liquidation_ratios affect bad debt creation during ETH/USD price dops.

## Methodology

Simulate an ETH/USD price drop of X%, then use RAI from Uniswap pools to settle resulting collateral auction and determine how much bad debt is created.

## Simulations

#### 1) Mainnet collateral amounts and c-ratios

#### 2) Mainnet collateral amounts and stochastic c-ratios

#### 3) Stochastic collateral amounts and stochastic c-ratios

#### 4) Gaussian Mixture model for collateral amounts and c-ratios


## Pessimistic Assumptions
### 1) All RAI for collateral auctions is purchased from market. No RAI-holding keepers exist
### 2) Overly expensive RAI market price.  Keepers get worse deal for RAI than expected and can buy less collateral at a profit.
####     - RAI is priced at redemption price after an ETH/USD drop -> expensive RAI
####     - No RAI market selling pressure when keepers purchase RAI -> expensive RAI

### 3) Saviours not considered
### 4) V2 Pool liquidity is reduced by 50% for all shocks
### 5) 35% OSM Responsiveness to avoid liquidation
### 6) Uses static surplus. Doesn't reduce surplus over time due to negative SF

## Optimistic Assumptions
### 1) Gas price is not considered
### 2) Keeper efficiency not considered

Fetch ETH/USD price, redemption price and all Reflexer SAFEs

In [ ]:
# calc how much liquidity is removed from rai_v2_pool by savior safes
# given: savior_safes, liq_ratio, shock

In [ ]:
# run a single sim
def run(orig_safes, saviour_safes, liq_ratio, shock, initial_surplus, verbose=False):
    
    #debt_adj =  MAINNET_LIQrai_v2O/ liq_ratio
    #safes = adjust_safe_debt_and_cratios(orig_safes, debt_adj, ETH_USD, REDEMPTION_PRICE)
    
    # Set eth shock price
    eth_shock_price = ETH_USD * (1 - shock)
    
    # update c-ratios
    safes = update_cratios(orig_safes, eth_shock_price, REDEMPTION_PRICE)
    
    if len(saviour_safes) > 0:
        saviour_safes = update_cratios(saviour_safes, eth_shock_price, REDEMPTION_PRICE)    
        # total syscoin removed from pool by saviours 
        
        syscoin_lp = saviour_safes[saviour_safes['cratio'] < liq_ratio]['lp_syscoin'].sum()
        rai_v2_pool_left = rai_v2_pool - syscoin_lp
    else:
        rai_v2_pool_left = rai_v2_pool 

    # Liquidate all critical SAFEs
    critical_collateral, critical_debt = liquidate_critical(safes, liq_ratio)
    
    #critical_collateral *= (1 - SAVIOUR_COLLATERAL)
    #critical_debt *= (1 - SAVIOUR_COLLATERAL)
       
    critical_collateral *= (1 - OSM_RESPONSIVENESS)
    critical_debt *= (1 - OSM_RESPONSIVENESS)
      
    if verbose:
        print(f"{critical_collateral=:.0f}, {critical_debt:.0f}, ratio {critical_collateral/critical_debt:.2f}")
    amount_to_raise = critical_debt * LIQ_PENALTY
    discount_collateral_price = eth_shock_price * DISCOUNT
    
    total_collateral_cost = critical_collateral * discount_collateral_price
    
    # Use v3 pool 
    
    # Assume rai market = redemption price and estimate slippage
    rai_v3_market_price = REDEMPTION_PRICE * 1.005 # Estimate slippage in V3 pool.

    # Use up to the entire pool to buy discounted collateral
    v3_collateral_bought = min((rai_v3_pool * REDEMPTION_PRICE)/discount_collateral_price, critical_collateral) 

    collateral_left = critical_collateral - v3_collateral_bought

    rai_v3_spent = v3_collateral_bought * discount_collateral_price / REDEMPTION_PRICE

    amount_raised_v3 = rai_v3_spent

    if verbose:
        print(f"{amount_raised_v3=:0f}")
        
    if collateral_left == 0:
        amount_deficit = critical_debt - amount_raised_v3

    else: # then use V2 pool

        amount_left_to_raise = amount_to_raise - rai_v3_spent
        if verbose:
            print(f"Using V2 pool. {amount_left_to_raise=:.0f}")
        collateral_left_cost = collateral_left * discount_collateral_price

        # max price to pay for RAI to make a profit on discounted collateral
        max_market_price = REDEMPTION_PRICE / DISCOUNT

        # Max amount to buy to put cost-basis at `max_market_price`
        how_much_v2_can_buy = 2 * buy_to_price(None, rai_v2_pool_left, goal_price=max_market_price, market_price=REDEMPTION_PRICE)

        # how is the pool affected when with this swap
        # This just confirms the expected final market price
        #delta_usd, delta_rai = get_output_price(how_much_v2_can_buy, usd_v2_pool, rai_v2_pool, trade_fee=0.003)
        # new market price
        #print(f"market price after buy {(usd_v2_pool + delta_usd)/(rai_v2_pool + delta_rai):.2f}")

        amount_raised_v2 = min(amount_left_to_raise, how_much_v2_can_buy)

        amount_deficit = critical_debt - amount_raised_v2 - amount_raised_v3

    if verbose:
        print(f"{amount_deficit=:2f}")
        
    return initial_surplus - amount_deficit

In [ ]:
# run many sims
def run_sims(safes, title='', sim_name='', verbose=False, plot=False):
    assert len(safes) == N_RUNS
    all_shocks = []
    all_surplus = []
    all_results = []
    
    for lr in LRS:
        # `debt_adj` is used to adjust individual SAFE debt to match expected values when liq_ratio is changed
        # This preserves safe debt as a multiplier of liq_ratio
        debt_adj =  MAINNET_LIQ_RATIO/ lr
        lr_safes = adjust_safe_debt_and_cratios(safes, debt_adj, ETH_USD, REDEMPTION_PRICE)
        
        for initial_surplus in SURPLUSES:
            config_shocks = []
            config_surpluses = []
            
            for i in range(N_RUNS):
                run_safes = lr_safes[i]
                lr_surpluses = []
                
                for s in SHOCKS:
                    surplus = run(run_safes, [], lr, s, initial_surplus)                    
                    config_shocks.append(s)
                    config_surpluses.append(surplus)
                    
                    if s == 0.0 and surplus < 0:
                        raise ValueError("negative surplus at zero shock")
                    
                    all_shocks.append(s)
                    all_surplus.append(surplus)
                    
                    if plot:
                        lr_surpluses.append(surplus)

                if plot:
                    plt.plot(SHOCKS, lr_surpluses, alpha=0.1)

            df = pd.DataFrame({'sim_name': sim_name, 'lr': lr, 'initial_surplus': initial_surplus, 
                               'shock': config_shocks,  'surplus': config_surpluses})
            all_results.append(df)

            for q in QUANTILES:
                if verbose:
                    df_q = df[['shock', 'surplus']].groupby('shock').quantile(1. - q).reset_index()
                    zero_q = df_q[df_q['surplus'] > 0].sort_values('surplus')

                    zero_debt_q = zero_q.iloc[0]['shock']
                    print(f"{lr}, {initial_surplus}, {q}, {zero_debt_q:.3f}")

                if plot:
                    df_100 = df[['shock', 'surplus']].groupby('shock').quantile(1.0).reset_index()
                    plt.plot(df_q['shock'], df_q['surplus'], ':')
                    plt.plot(df_100['shock'], df_100['surplus'], ':')
                    plt.fill_between(SHOCKS, 
                                df_100['surplus'], 
                                df_q['surplus'], alpha=0.4,label=f"{q*100}% confidence")

            if plot:
                df_mean = df[['shock', 'surplus']].groupby('shock').mean().reset_index()
                plt.plot(df_mean['shock'], df_mean['surplus'])
                plt.title(f'{title}\n{lr=}, {initial_surplus=} {N_RUNS=}', fontsize=14)
                plt.xlabel('ETH/USD Shock (0.5 = 50% drop)', fontsize=16)
                plt.ylabel('Surplus', fontsize=16)
                plt.grid(True)
                plt.legend(loc='lower left')
                plt.ylim([-1E7,2E6])
                plt.show()
                
    return pd.concat(all_results)
    #return pd.DataFrame(results, columns=['sim_name', 'lr', 'initial_surplus', 'q', 'final_surplus'])


In [ ]:
# run many sims over multiple subsequent shocks
def run_sims_iter(safes, n_shocks=5, title='', sim_name='', verbose=False, plot=False):
    assert len(safes) == N_RUNS
    all_shocks = []
    all_surplus = []
    all_results = []
    
    for lr in LRS:
        # `debt_adj` is used to adjust individual SAFE debt to match expected values when liq_ratio is changed
        # This preserves safe debt as a multiplier of liq_ratio
        debt_adj =  MAINNET_LIQ_RATIO/ lr
        lr_safes = [adjust_safe_debt_and_cratios(safe, debt_adj, ETH_USD, REDEMPTION_PRICE) for safe in safes]
        for initial_surplus in SURPLUSES:
            config_shocks = []
            config_surpluses = []
            
            for i in range(N_RUNS):
                run_safes = lr_safes[i]
                lr_surpluses = []               
                for s in SHOCKS:
                    total_shocks_run = 0
                    surplus = initial_surplus
                    while total_shocks_run < n_shocks:
                        surplus = run(run_safes, [], lr, s, surplus)
                        total_shocks_run += 1 
                            
                    config_shocks.append(s)
                    config_surpluses.append(surplus)
                    
                    if s == 0.0 and surplus < 0:
                        raise ValueError("negative surplus at zero shock")
                    
                    all_shocks.append(s)
                    all_surplus.append(surplus)
                    
                    if plot:
                        lr_surpluses.append(surplus)

                if plot and N_RUNS < 100: # Plotting too many runs is slow
                    plt.plot(SHOCKS, lr_surpluses, alpha=0.1)

            df = pd.DataFrame({'sim_name': sim_name, 'lr': lr, 'initial_surplus': initial_surplus, 
                               'shock': config_shocks,  'surplus': config_surpluses})
            
            all_results.append(df)
        
            for q in QUANTILES:
                if verbose:
                    df_q = df[['shock', 'surplus']].groupby('shock').quantile(1. - q).reset_index()
                    zero_q = df_q[df_q['surplus'] > 0].sort_values('surplus')

                    zero_debt_q = zero_q.iloc[0]['shock']
                    print(f"{sim_name=}, {lr=}, {initial_surplus=}, {q=}, {zero_debt_q=:.3f}")

                if plot and len(QUANTILES) <= 10:
                    df_100 = df[['shock', 'surplus']].groupby('shock').quantile(1.0).reset_index()
                    plt.plot(df_q['shock'], df_q['surplus'], ':')
                    plt.plot(df_100['shock'], df_100['surplus'], ':')
                    plt.fill_between(SHOCKS, 
                                df_100['surplus'], 
                                df_q['surplus'], alpha=0.4,label=f"{q*100}% confidence")

            if plot:
                df_mean = df[['shock', 'surplus']].groupby('shock').mean().reset_index()
                plt.plot(df_mean['shock'], df_mean['surplus'])
                plt.title(f'{title}\n{lr=}, {initial_surplus=} {N_RUNS=}', fontsize=14)
                plt.xlabel('ETH/USD Shock (0.5 = 50% drop)', fontsize=16)
                plt.ylabel('Surplus', fontsize=16)
                plt.grid(True)
                plt.legend(loc='lower left')
                plt.ylim([-1E7,2E6])
                plt.show()
                
    return pd.concat(all_results)


In [ ]:
# run many sims over multiple subsequent shocks
def run_sims_past_shocks(safes, past_shocks, title='', sim_name='', verbose=False, plot=False):
    assert len(safes) == N_RUNS
    all_shocks = []
    all_surplus = []
    all_results = []
    
    for lr in LRS:
        # `debt_adj` is used to adjust individual SAFE debt to match expected values when liq_ratio is changed
        # This preserves safe debt as a multiplier of liq_ratio
        debt_adj =  MAINNET_LIQ_RATIO/ lr
        lr_safes = [adjust_safe_debt_and_cratios(safe, debt_adj, ETH_USD, REDEMPTION_PRICE) for safe in safes]
        for initial_surplus in SURPLUSES:
            config_shocks = []
            config_surpluses = []
            
            for i in range(N_RUNS):
                run_safes = lr_safes[i]
                lr_surpluses = [] 
                surplus = initial_surplus
                for s in past_shocks:
                    surplus = run(run_safes, [], lr, s, surplus)
                    #print(f"{s=}, {surplus=}")
                            
                config_surpluses.append(surplus)                   
                    
                all_shocks.append(s)
                all_surplus.append(surplus)



            df = pd.DataFrame({'sim_name': sim_name, 'lr': lr, 'initial_surplus': initial_surplus, 
                               'surplus': config_surpluses})           
            all_results.append(df)

            for q in QUANTILES:
                if verbose:
                    df_q = df['surplus'].quantile(1. - q)
                

                    print(f"{sim_name=}, {lr=}, {initial_surplus=}, {q=}, {df_q=:.3f}")


            if plot:
                df_mean = df[['shock', 'surplus']].groupby('shock').mean().reset_index()
                plt.plot(df_mean['shock'], df_mean['surplus'])
                plt.title(f'{title}\n{lr=}, {initial_surplus=} {N_RUNS=}', fontsize=14)
                plt.xlabel('ETH/USD Shock (0.5 = 50% drop)', fontsize=16)
                plt.ylabel('Surplus', fontsize=16)
                plt.grid(True)
                plt.legend(loc='lower left')
                plt.ylim([-1E7,2E6])
                plt.show()
                
    return pd.concat(all_results)


# Sim #1: Mainnet SAFE Collateral Amounts and C-ratios

In [ ]:
SHOCKS = np.round(np.linspace(min_shock, max_shock, n_shocks), 4)

for lr in LRS:    
    lr_surpluses = []
    debt_adj =  MAINNET_LIQ_RATIO/ lr
    safes = adjust_safe_debt_and_cratios(non_saviour_safes, debt_adj, ETH_USD, REDEMPTION_PRICE)
    saviour_safes_adj = adjust_safe_debt_and_cratios(saviour_safes, debt_adj, ETH_USD, REDEMPTION_PRICE)
    for s in SHOCKS:
        #surplus = run(safes, saviour_safes_adj, lr, s, initial_surplus=INITIAL_SURPLUS, verbose=False)
        
        surplus = run(safes, [], lr, s, initial_surplus=INITIAL_SURPLUS, verbose=False)
        #print(f"{s=}, {surplus=}")
        #print(f"{lr=}, {s=}, {surplus=}")
        lr_surpluses.append(surplus)
    plt.plot(SHOCKS, lr_surpluses, label=lr)
    
plt.legend()
plt.title('Resulting debt for different liquidation ratios and ETH/USD shocks, given current Reflexer SAFE statuses\n', fontsize=14)
plt.xlabel('ETH/USD Shock (0.5 = 50% drop)', fontsize=16)
plt.ylabel('Surplus', fontsize=16)
plt.grid(True)
plt.show()

## Compare Mainnet Saviour Safes to Non-Saviour

In [ ]:
print(f"percent of mainnet collateral w/ saviours: {saviour_safes['collateral'].sum() /orig_safes['collateral'].sum():.2f}")

In [ ]:
saviour_safes = update_cratios(saviour_safes, ETH_USD, REDEMPTION_PRICE)
non_saviour_safes = update_cratios(non_saviour_safes, ETH_USD, REDEMPTION_PRICE)

orig_safes = update_cratios(orig_safes, ETH_USD, REDEMPTION_PRICE)
bins = np.linspace(1.45, orig_safes['cratio'].max(), 50)

saviour_safes['cratio'].hist(bins=bins, alpha=0.5, color='red', label='saviour')

non_saviour_safes['cratio'].hist(bins=bins, color='blue', alpha=0.5, label='non-saviour')

orig_safes['cratio'].hist(bins=bins, color='green', alpha=0.2, label='all safes')

plt.title('SAFE C-ratios', fontsize=18)
plt.xlabel('C-ratio', fontsize=16)
plt.ylabel('Number of SAFEs', fontsize=16)
plt.legend()
plt.grid(False)
plt.show()

# Sim #2: Mainnet SAFE Collateral Amounts with Stochastic C-ratios

### Examine mainnet c-ratios

In [ ]:
non_saviour_safes = update_cratios(non_saviour_safes, ETH_USD, REDEMPTION_PRICE)
non_saviour_safes['cratio'].hist(bins=100)
plt.title('Mainnet non-saviour SAFE C-ratios', fontsize=18)
plt.xlabel('C-ratio', fontsize=16)
plt.ylabel('Number of SAFEs', fontsize=16)
plt.grid(False)
plt.show()

### Fit many distributions to mainnnet c-ratios

In [ ]:
fitter_cratio = Fitter(non_saviour_safes['cratio'].values)          
fitter_cratio.fit()
fitter_cratio.summary()

### Pick the best distribution and generate some c-ratios

In [ ]:
# best_cratio = fitter_cratio.get_best(method = 'sumsquare_error')
cratio_params = fitter_cratio.fitted_param["mielke"]

In [ ]:
cratios_r = ss.mielke.rvs(*cratio_params, size=len(non_saviour_safes)*2)
cratios_r = cratios_r[cratios_r > MAINNET_LIQ_RATIO]

df = pd.DataFrame(cratios_r, columns=['cratio'])
df = df.head(len(non_saviour_safes))

df['cratio'].hist(bins=100, alpha=0.5, color='red', label='generated')

non_saviour_safes['cratio'].hist(bins=100, color='blue', alpha=0.5, label='original')

plt.title('Non-saviour SAFE C-ratios', fontsize=18)
plt.xlabel('C-ratio', fontsize=16)
plt.ylabel('Number of SAFEs', fontsize=16)
plt.legend()
plt.grid(False)
plt.show()

# Sim #2: Mainnet SAFE collateral amounts with stochastic c-ratios

### Prepare SAFEs

In [ ]:
all_cratios = []
for _ in range(N_RUNS):
    all_cratios.append(ss.mielke.rvs(*cratio_params, size=len(non_saviour_safes)*2))

cratio_rv = ss.mielke(*cratio_params)

all_tmp_safes = [pd.DataFrame(cr, columns=['cratio']) for cr in all_cratios]

final_sim2_safes = []
for tmp_safes in all_tmp_safes:
    tmp_safes = tmp_safes[tmp_safes['cratio'] > MAINNET_LIQ_RATIO]        
    tmp_safes = tmp_safes.head(len(non_saviour_safes))
    assert len(tmp_safes) == len(non_saviour_safes)

    tmp_safes['collateral'] = non_saviour_safes['collateral']
    tmp_safes['debt'] = tmp_safes['collateral'] * ETH_USD / REDEMPTION_PRICE / tmp_safes['cratio']

    # Consider debt ceiling, adding more possible debt
    added_debt = 0
    while added_debt < extra_debt:
        new_cr = cratio_rv.rvs(size=1)[0]
        while new_cr < MAINNET_LIQ_RATIO: 
            new_cr = cratio_rv.rvs(size=1)[0]
            
        new_coll = non_saviour_safes['collateral'].sample().iloc[0]

        new_debt = min(new_coll * ETH_USD / REDEMPTION_PRICE / new_cr, extra_debt - added_debt)
        added_debt += new_debt
        tmp_safes = tmp_safes.append({'cratio': new_cr, 'collateral': new_coll, 'debt': new_debt}, ignore_index=True)
        
    final_sim2_safes.append(tmp_safes)        
    assert added_debt == extra_debt

# Sim #3: Stochastic SAFE collateral amounts and stochastic c-ratios

### Fit many distributions to mainnnet collateral amounts

In [ ]:
fitter_coll = Fitter(np.log(non_saviour_safes['collateral'].values))        
fitter_coll.fit()
fitter_coll.summary()

### Pick the best distribution and generate some collateral amounts

In [ ]:
#best_coll = fitter_coll.get_best(method = 'sumsquare_error')
coll_params = fitter_coll.fitted_param["skewcauchy"]

In [ ]:
colls_r = ss.skewcauchy.rvs(*coll_params, size=len(non_saviour_safes)*2)

colls_r = np.exp(colls_r)

# Collateral

df = pd.DataFrame(colls_r, columns=['collateral'])
df = df[(df['collateral'] > 0) & (df['collateral'] < non_saviour_safes['collateral'].max()) & (tmp_safes['cratio'] > MAINNET_LIQ_RATIO)]
df = df.head(len(non_saviour_safes))

df['collateral'].hist(bins=100, alpha=0.5, color='red', label='generated')

non_saviour_safes['collateral'].hist(bins=100, color='blue', alpha=0.5, label='original')
#plt.xlim([0, 1000])
plt.title('SAFE Collateral', fontsize=18)
plt.xlabel('Collateral', fontsize=16)
plt.ylabel('Number of SAFEs', fontsize=16)
plt.legend()
plt.grid(False)
plt.show()

### Sim #3: Prepare SAFEs

In [ ]:
all_cratios = []
for _ in range(N_RUNS):
    all_cratios.append(ss.mielke.rvs(*cratio_params, size=len(non_saviour_safes)*2))
    
all_collaterals = []
for _ in range(N_RUNS):
    all_collaterals.append(np.exp(ss.skewcauchy.rvs(*coll_params, size=len(non_saviour_safes)*2)))
    
cratio_rv = ss.mielke(*cratio_params)
coll_rv = ss.skewcauchy(*coll_params)

all_tmp_safes = [pd.DataFrame({'collateral': coll, 'cratio': cr})  for coll, cr in zip(all_collaterals, all_cratios)]

final_sim3_safes = []
for tmp_safes in all_tmp_safes:
    tmp_safes = tmp_safes[(tmp_safes['collateral'] > 0) & (tmp_safes['collateral'] < non_saviour_safes['collateral'].max()) & (tmp_safes['cratio'] > MAINNET_LIQ_RATIO)]
    tmp_safes = tmp_safes.head(len(non_saviour_safes))
    assert len(tmp_safes) == len(non_saviour_safes)
    tmp_safes['debt'] = tmp_safes['collateral'] * ETH_USD / REDEMPTION_PRICE / tmp_safes['cratio']
    added_debt = 0
    while added_debt < extra_debt:
        # Sample c-ratios and collateral amounts
        new_cr = cratio_rv.rvs(size=1)[0]
        while new_cr < MAINNET_LIQ_RATIO: 
            new_cr = cratio_rv.rvs(size=1)[0]
            
        new_coll = np.exp(coll_rv.rvs(size=1))[0]
                 
        # calculate debt
        new_debt = min(new_coll * ETH_USD / REDEMPTION_PRICE / new_cr, extra_debt - added_debt)
        added_debt += new_debt
                
        # Add new SAFE
        tmp_safes = tmp_safes.append({'cratio': new_cr, 'collateral': new_coll, 'debt': new_debt}, ignore_index=True)
        
    assert added_debt == extra_debt
    final_sim3_safes.append(tmp_safes)
    
    

# Sim #4: Gaussian Mixture Model for SAFE collateral amounts and c-ratios

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm
from sklearn import mixture

# Create a Gaussian Mixture Model with one component
clf = mixture.GaussianMixture(n_components=1, covariance_type='full')

safes = non_saviour_safes.copy()

# Transform collateral to make it normal
#pt = PowerTransformer()
#safes['collateral'] = pt.fit_transform(safes['collateral'].values.reshape(-1,1))
safes['collateral'] = np.log(safes['collateral'])

"""
safes['collateral'].hist(bins=100, color='blue', alpha=0.5, label='original')
plt.title('Transformed collateral')
plt.legend()
plt.grid(False)
plt.show()
"""

# Fit
clf.fit(safes[['collateral', 'cratio']])

# display predicted scores by the model as a contour plot
x = np.linspace(0, 50.)
y = np.linspace(0, 20.)
X, Y = np.meshgrid(x, y)
XX = np.array([X.ravel(), Y.ravel()]).T
Z = -clf.score_samples(XX)
Z = Z.reshape(X.shape)

CS = plt.contour(X, Y, Z, norm=LogNorm(vmin=1.0, vmax=1000.0),
                 levels=np.logspace(0, 3, 10))
CB = plt.colorbar(CS, shrink=0.8, extend='both')

plt.scatter(safes['collateral'], safes['cratio'], .8, color='blue')

#plt.title('Negative log-likelihood predicted by a GMM')
plt.axis('tight')
plt.xlabel('Log Collateral', fontsize=16)
plt.ylabel('C-ratio', fontsize=16)
plt.show()


In [ ]:
# C-ratios
sample = np.random.multivariate_normal(clf.means_[0], clf.covariances_[0], len(non_saviour_safes)*2)
df = pd.DataFrame(sample, columns=['collateral', 'cratio'])
df = df[df['cratio'] > MAINNET_LIQ_RATIO]
df = df.head(len(non_saviour_safes))

bins = np.linspace(1.45, 14, 50)
df['cratio'].hist(bins=bins, alpha=0.5, color='red', label='generated')
non_saviour_safes['cratio'].hist(bins=bins, color='blue', alpha=0.5, label='original non-saviour-safes')

plt.title('SAFE C-ratios', fontsize=18)
plt.xlabel('C-ratio', fontsize=16)
plt.ylabel('Number of SAFEs', fontsize=16)
plt.legend()
plt.grid(False)
plt.show()

# Collateral
sample = np.random.multivariate_normal(clf.means_[0], clf.covariances_[0], len(non_saviour_safes)*2)
df = pd.DataFrame(sample, columns=['collateral', 'cratio'])
df = df[df['cratio'] > MAINNET_LIQ_RATIO]
df = df.head(len(non_saviour_safes))

df['collateral'] = np.exp(df['collateral'])

bins = np.linspace(0, 1000, 50)
df['collateral'].hist(bins=bins, alpha=0.5, color='red', label='generated')

non_saviour_safes['collateral'].hist(bins=bins, alpha=0.5, color='blue', label='original')
plt.xlim([0, 1000])
plt.title('non-saviour SAFE Collateral', fontsize=18)
plt.xlabel('Collateral', fontsize=16)
plt.ylabel('Number of SAFEs', fontsize=16)
plt.legend()
plt.grid(False)
plt.show()

## Sim #4: Prepare SAFEs

In [ ]:
# Generate a population of SAFEs for each run

# draw random collateral and c-ratio values from distribution
all_gaussian_safes = [np.random.multivariate_normal(clf.means_[0], clf.covariances_[0], len(non_saviour_safes)*2) for _ in range(N_RUNS)]

# Intialize SAFEs with random values
all_safes = [pd.DataFrame(gaussian_safe, columns=['collateral', 'cratio']) for gaussian_safe in all_gaussian_safes]

# Filter out unacceptable SAFEs
all_tmp_safes = []

for tmp_safes in all_safes:
    tmp_safes['collateral'] = np.exp(tmp_safes['collateral'])
    tmp_safes = tmp_safes[(tmp_safes['collateral'] > 0) & (tmp_safes['cratio'] > MAINNET_LIQ_RATIO)]
    
    tmp_safes = tmp_safes.head(len(non_saviour_safes))
    assert len(tmp_safes) == len(non_saviour_safes)
    
    tmp_safes['debt'] = tmp_safes['collateral'] * ETH_USD / REDEMPTION_PRICE / tmp_safes['cratio']    
    all_tmp_safes.append(tmp_safes)
    
# Draw more random collateral and c-ratio values
all_extra_gaussian_safes = [np.random.multivariate_normal(clf.means_[0], clf.covariances_[0], 1000) for _ in range(N_RUNS)]

# Generate more SAFEs to fill up to debt ceiling
final_sim4_safes = []
for idx, gaussian_safes in enumerate(all_extra_gaussian_safes):
    new_safes = pd.DataFrame(gaussian_safes, columns=['collateral', 'cratio'])
    new_safes['collateral'] = np.exp(new_safes['collateral'])
    
     # Filter out unacceptable samples
    new_safes = new_safes[(new_safes['collateral'] > 0) & (new_safes['cratio'] > MAINNET_LIQ_RATIO)]
    
    # Calculate debt
    new_safes['debt'] = new_safes['collateral'] * ETH_USD / REDEMPTION_PRICE / new_safes['cratio']
    
    # Only use enough SAFEs to fill `extra_debt`
    new_safes['total_debt'] = new_safes['debt'].cumsum()        
    final_safes = new_safes[new_safes['total_debt'] <= extra_debt]
    assert len(final_safes) > 0
    
    final_safes.drop('total_debt', axis='columns')
    
    tmp_safes = pd.concat([all_tmp_safes[idx], final_safes])
    final_sim4_safes.append(tmp_safes)

## Run All Sims

In [ ]:
def process_results(df):
    for q in QUANTILES:
        print(f"{q=}")
        df_q = df[['lr', 'initial_surplus', 'shock', 'surplus']].groupby(['lr', 'initial_surplus', 'shock']).quantile(1. - q).reset_index()       
        df_q = df_q[df_q['surplus'] > 0]
        df_min = df_q.loc[df_q.groupby(['lr', 'initial_surplus'])['surplus'].idxmin()]
        print(df_min.head(100))            

## Sims past shocks

In [ ]:
eth_shocks = pd.read_csv('binance_eth_shocks.csv')
smallest_shock = 0.02
past_shocks = eth_shocks.query(f'pct_change <= -{smallest_shock}')['pct_change']
past_shocks = [-x for x in past_shocks]


final_sim2_safes = final_sim2_safes[:N_RUNS]
final_sim3_safes = final_sim3_safes[:N_RUNS]
final_sim4_safes = final_sim4_safes[:N_RUNS]

#N_RUNS = 100
#LRS = [1.20, 1.30, 1.35, 1.40]
SURPLUSES = [INITIAL_SURPLUS]

# Analysis Parameters
QUANTILES = [0.90, 0.95, 0.99]

PLOT = False
VERBOSE = True

In [ ]:
VERBOSE=True

#sim2_results = run_sims_past_shocks(final_sim2_safes, past_shocks, title='', sim_name='sim2', verbose=VERBOSE, plot=PLOT)
#sim3_results = run_sims_past_shocks(final_sim3_safes, past_shocks, title='', sim_name='sim3', verbose=VERBOSE, plot=PLOT)
sim4_results = run_sims_past_shocks(final_sim4_safes, past_shocks, title='', sim_name='sim4', verbose=VERBOSE, plot=PLOT)
#all_results = pd.concat([sim2_results, sim3_results, sim4_results])
#process_results(all_results)

In [ ]:
smallest_shock = 0.02
past_shocks = df_filter.query(f'pct_change <= -{smallest_shock}')['pct_change']
len(past_shocks)

In [ ]:
N_RUNS = 1000
LRS = [1.20, 1.25, 1.30, 1.35, 1.40]
SURPLUSES = [INITIAL_SURPLUS, 1E6]

min_shock = 0.00
max_shock = 0.6
n_shocks = 61
SHOCKS = np.round(np.linspace(min_shock, max_shock, n_shocks), 4)
TOTAL_SHOCKS = [1,2,3]

# Analysis Parameters
QUANTILES = [0.90, 0.95, 0.99]

PLOT = False
VERBOSE = True

## Sims Iterations

In [ ]:
N_RUNS = 10
VERBOSE=True
for n_shocks in TOTAL_SHOCKS:
    print(f"{n_shocks=}")
    sim2_results = run_sims_iter(final_sim2_safes, n_shocks=n_shocks, title='', sim_name='sim2', verbose=VERBOSE, plot=PLOT)
    sim3_results = run_sims_iter(final_sim3_safes, n_shocks=n_shocks, title='', sim_name='sim3', verbose=VERBOSE, plot=PLOT)
    sim4_results = run_sims_iter(final_sim4_safes, n_shocks=n_shocks, title='', sim_name='sim4', verbose=VERBOSE, plot=PLOT)
    all_results = pd.concat([sim2_results, sim3_results, sim4_results])
    process_results(all_results)